- Load a dataset

- Make it as a iterator

- Load a tokeniser

- Write a processing function 

- Map it to the dataset 

- Create a new tokeniser

- Train it with the dataset

- Write the post processing function

- Run the evaluation 

In [1]:
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    AutoModelForMaskedLM,
)
from datasets import load_dataset, load_metric

In [3]:
from torch.utils.data import DataLoader, Dataset

In [4]:
from torchtext.datasets import IMDB

ModuleNotFoundError: No module named 'torchtext'

In [ ]:
wikiset

In [ ]:
# model = 'distilgpt2'
model = 'distilroberta-base'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model)

In [30]:
tokenizer.model_max_length

512

In [ ]:
def pre_process(row):
    return tokenizer(row['text'])

In [ ]:
wikiset_tokenised = wikiset.map(pre_process, batched=True)

In [ ]:
wikiset_tokenised = wikiset_tokenised.remove_columns(['text'])

In [ ]:
wikiset_tokenised

In [ ]:
block_size = 128


def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of block_size.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
lm_dataset = wikiset_tokenised.map(group_texts, batched=True)

In [ ]:
lm_dataset

In [ ]:
tokenizer.pad_token = tokenizer.eos_token
datacollator = DataCollatorForLanguageModeling(tokenizer, mlm=False)
datacollator_mlm = DataCollatorForLanguageModeling(tokenizer, 
                                                   mlm=True, mlm_probability=0.15)

In [ ]:
targs = TrainingArguments(
    output_dir="/home/kamal/training_files/prac/",
    eval_steps=500,
    evaluation_strategy='steps',
    per_device_eval_batch_size=8,
    per_device_train_batch_size=8,
    num_train_epochs=2,
    push_to_hub=False,
    report_to="none",
    learning_rate=2e-5,
    weight_decay=0.01
)

In [47]:
import torch
del model_wt
torch.cuda.empty_cache()

In [ ]:
# model_wt = AutoModelForCausalLM.from_pretrained(model, is_decoder=True)
model_wt = AutoModelForMaskedLM.from_pretrained(model)

In [ ]:
trainer = Trainer(
    model=model_wt,
    args=targs,
    data_collator=datacollator,
    train_dataset=lm_dataset['train'],
    eval_dataset=lm_dataset['test'],
)

In [45]:
trainer.train()

Step,Training Loss,Validation Loss
500,5.057700,4.320317
1000,4.403100,4.173672
1500,4.283600,4.095723
2000,4.213000,4.049093
2500,4.147300,4.019415
3000,4.048800,4.003908
3500,4.030000,3.984876
4000,4.016000,3.975043
4500,3.993700,3.967570


Checkpoint destination directory /home/kamal/training_files/prac/checkpoint-500 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory /home/kamal/training_files/prac/checkpoint-1000 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory /home/kamal/training_files/prac/checkpoint-1500 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory /home/kamal/training_files/prac/checkpoint-2000 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory /home/kamal/training_files/prac/checkpoint-2500 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory /home/kamal/training_files/prac/checkpoint-3000 already exists and is non-empty.Saving will proceed but saved results may be invalid.

TrainOutput(global_step=4812, training_loss=4.227675947663394, metrics={'train_runtime': 357.2386, 'train_samples_per_second': 107.726, 'train_steps_per_second': 13.47, 'total_flos': 1275954401221632.0, 'train_loss': 4.227675947663394, 'epoch': 2.0})